In [1]:
import torch
import torch.nn as nn
import time
import os

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('full_model.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=25, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=25, out_features=5, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([25, 36])
l1.bias torch.Size([25])
l2.weight torch.Size([5, 25])
l2.bias torch.Size([5])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {-0.33779868483543396, 0.054830364882946014, 0.06808087974786758, -0.005753322038799524, 0.13115806877613068, 0.15108120441436768, -0.0372389554977417, -0.0006150839617475867, -0.05548153817653656, -0.24002790451049805, -0.11639934033155441, -0.15681229531764984, -0.15068449079990387, -0.156447172164917, 0.015133440494537354, 0.07443244010210037, 0.08039846271276474, -0.0374448224902153, -0.12388001382350922, -0.20451875030994415, 0.19782447814941406, 0.09124812483787537, -0.2473715990781784, 0.23850375413894653, 0.020914148539304733};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {0.27175644040107727, -0.09232676029205322, -0.08046212047338486, 0.0830453485250473, -0.18527114391326904};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{0.21053332090377808, -0.25532910227775574, -0.03737613186240196, 0.26100999116897583, 0.08764150738716125, 0.3081146478652954, 0.345599502325058, -0.48723652958869934, 0.37817442417144775, 0.4439048767089844, 0.5551561713218689, 0.3852725327014923, 0.07341918349266052, 0.05755910649895668, 0.24518904089927673, -0.005021474789828062, 0.1873893439769745, 0.052165161818265915, 0.11212225258350372, -0.011975040659308434, -0.13421247899532318, 0.0648156926035881, 0.21822965145111084, -0.14599163830280304, -0.13544096052646637, -0.12034083902835846, -0.00848313793540001, -0.08689683675765991, 0.018503354862332344, 0.06450486928224564, -0.0007688446785323322, -0.006668309681117535, -0.3596526086330414, 0.0223139189183712, -0.1866263449192047, 0.15415436029434204},
{0.06660101562738419, -0.14873988926410675, 0.4965701997280121, 0.0890885517001152, -0.03886343166232109, 0.17456930875778198, 0.1315522938966751, -0.10348666459321976, -0.10134986

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{-0.20815961062908173, -0.047319479286670685, 0.22636938095092773, 0.22530415654182434, -0.025788752362132072, -0.010448458604514599, 0.013796532526612282, -0.10144349187612534, 0.04587579146027565, -0.14906802773475647, 0.014113799668848515, -0.08183663338422775, -0.13637320697307587, 0.024219641461968422, 0.13980723917484283, 0.23812642693519592, -0.010126009583473206, 0.04079675301909447, 0.15550167858600616, 0.09702173620462418, 0.07097985595464706, -0.004818076267838478, 0.04082874208688736, 0.12305928021669388, -0.25404974818229675},
{0.19213613867759705, -0.00895760115236044, -0.0037500890903174877, -0.19173912703990936, 0.15896981954574585, 0.17300164699554443, 0.15158076584339142, -0.07427630573511124, -0.07557936012744904, 0.14319592714309692, -0.03661399334669113, -0.1290605664253235, 0.12795783579349518, -0.18630924820899963, -0.17122507095336914, -0.20273537933826447, 0.09804308414459229, -0.06047939881682396, 0.034721821

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [12]:
DATA_DIR = 'data'
RAW = DATA_DIR + '/raw'
NUM_FEATURES = 36

In [13]:
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
X_test = test_df.drop('action', axis='columns')
y_test = test_df['action']

In [14]:
data = [torch.Tensor(X_test.iloc[x].astype(float)) for x in range(5)]

In [15]:
data

[tensor([ 3.1000e-01, -5.8000e-01, -1.2000e-01,  1.7015e-01,  1.1500e-01,
          1.2400e-01,  1.4300e+00,  1.0000e-02,  4.4075e-01,  4.5913e-01,
          5.8000e-01,  3.9505e-01,  4.0000e-02, -2.0600e+00, -3.2475e-01,
          4.1448e-01,  3.3000e-01,  3.0715e-01,  8.6600e+01, -2.4400e+02,
         -1.7197e+01,  6.3462e+01,  1.6583e+01,  3.8369e+01,  2.7284e+02,
         -2.7223e+02, -5.0725e-01,  9.3333e+01,  3.0083e+01,  5.1365e+01,
          9.4030e+01, -2.1819e+02, -2.2205e+01,  6.7730e+01,  2.2837e+01,
          4.5491e+01]),
 tensor([ 1.7500e+00, -1.2000e+00, -1.8750e-02,  5.2591e-01,  3.0500e-01,
          3.6687e-01,  2.2600e+00, -3.5000e-01,  5.8400e-01,  8.7613e-01,
          1.0375e+00,  7.5660e-01,  4.9000e-01, -9.1000e-01, -3.9500e-02,
          2.5641e-01,  1.5500e-01,  1.7530e-01,  1.5072e+02, -2.7449e+02,
         -6.0310e+00,  8.8101e+01,  2.7085e+01,  4.9556e+01,  1.1655e+02,
         -1.2804e+02,  1.3707e+01,  4.0749e+01,  3.1570e+01,  2.7034e+01,
          2.85

In [16]:
with torch.no_grad():
    for d in data:
        start_time = time.time()
        output = model(d)
        action = np.argmax(output)
#         print(f"Inference time: {time.time() - start_time}")    
        print(action)

tensor(3)
tensor(1)
tensor(1)
tensor(4)
tensor(2)


In [17]:
for x in range(5):
    print(y_test[x])

3
1
1
0
2


## Compare software-hardware running time and accuracy

In [18]:
# 1 sample

with torch.no_grad():
    start_time = time.time()

    data = torch.Tensor(X_test.iloc[0].astype(float))
    output = model(data)
    action = np.argmax(output)

    end_time = time.time()
    print(f"Prediction: {action}; Ground truth value: {y_test[0]}")
    print(f"Inference time: {((end_time - start_time) * 1000):.4f} ms") 

Prediction: 3; Ground truth value: 3
Inference time: 0.8259 ms


In [19]:
# whole test set (or dev + test set used in training)

with torch.no_grad():
    start_time = time.time()
    
    num_correct = 0
    num_samples = 0

    for i in range(len(X_test[:])):
        data = torch.Tensor(X_test.iloc[i].astype(float))
        output = model(data)
        action = np.argmax(output)
        num_samples += 1
        num_correct += (action == y_test[i])

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")    

Accuracy: 94.23%
Inference time: 0.0266 s


In [20]:
len(X_test)

52